In [2]:
import numpy as np
import pandas as pd
import scipy.io
from scipy import stats
import random
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
import sklearn

/Users/Me/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [17]:
# Training data generation

def generateTrain(file, n=None):
    # read in dataset and slice
    rawData = np.array(pd.read_csv(file, sep=",", header=None).values[1:])
    label = rawData[:, -1]
    data = rawData[:, :-1]
    if n != None:
        data = data[:n, :]  # Note: only use n lines
    # construct train feature matirx
    train = np.zeros((data.shape[0], 20))
    # distance ratio features
    for i in range(5):
        denominator = (data[:, 8*(i+1) - 1] - data[:, 0])**2 + (data[:, 8*(i+1)] - data[:, 1])**2  # distance from tip to palm
        for j in range(3):
            numerator = (data[:, 8*i + 2*j + 1] - data[:, 0])**2 + (data[:, 8*i + 2*j + 2] - data[:, 1])**2  # distance from root/mid1/mid2 to palm
            ratio = np.sqrt(numerator) / np.sqrt(denominator)
            train[:, i*3 + j] = ratio
            # train[:, i*3 + j] = ratio * 10  # 10 times to make more spearable?
    # finger number feature
    for i in range(len(train)):
        temp = train[i, :]
        train[i, 15] = sum(temp[3 : 15 : 4] <= 1) * 10  # stretch finger number, weighted by 10
    # angle features
    for i in range(4):  # four pairs
        x1 = np.array([data[:, 8*(i+1) - 1] - data[:, 0], data[:, 8*(i+1)] - data[:, 1]], dtype=np.float32).T
        x2 = np.array([data[:, 8*(i+2) - 1] - data[:, 0], data[:, 8*(i+2)] - data[:, 1]], dtype=np.float32).T
        cos = np.sum(x1*x2, axis=1) / (np.sqrt(np.sum(x1**2, axis=1)) * np.sqrt(np.sum(x2**2, axis=1)))  # caculate cos(theta)
        # when zero angle, it is possible
        cos[cos > 1] = 1
        cos[cos < -1] = -1
        train[:, 16 + i] = (np.arccos(cos) / np.pi * 180)
        # train[:, 16 + i] = (np.arccos(cos) / np.pi * 180)**2  # Note: use quadratic here

    # return revised matrix
    return train, label

In [7]:
training_df = pd.read_csv('/Users/Me/Desktop/assistive_mouse/threeClass.csv')
#training_df = pd.read_csv('/Users/Me/Desktop/assistive_mouse/data.txt', sep=",")
training_df[["42"]] = training_df[["42"]].astype("int")
X = training_df.drop(columns =["42"])

y = training_df[["42"]].astype(int)
training_df.columns = ['wrist_x', 'wrist_y', 
                       "side_x", "side_y",
                       
                       "thumb_base_x", "thumb_base_y", 
                       "thumb_middle_x", "thumb_middle_y",
                       "thumb_tip_x", "thumb_tip_y",
                       
                       "index_base_x", "index_base_y",
                       "index_low_x", "index_low_y",
                       "index_mid_x", "index_mid_y",
                       "index_tip_x", "index_tip_y",
                       
                       "middle_base_x", "middle_base_y", 
                       "middle_low_x", "middle_low_y",
                       "middle_mid_x", "middle_mid_y",
                       "middle_tip_x", "middle_tip_y",
                      
                       "ring_base_x", "ring_base_y",
                       "ring_low_x", "ring_low_y",
                       "ring_mid_x", "ring_mid_y",
                       "ring_tip_x", "ring_tip_y",
                      
                       "pinky_base_x", "pinky_base_y",
                       "pinky_low_x", "pinky_low_y",
                       "pinky_mid_x", "pinky_mid_y",
                       "pinky_tip_x", "pinky_tip_y", 
                       
                       "labels"]
display(training_df.tail(5))

,wrist_x,wrist_y,side_x,side_y,thumb_base_x,thumb_base_y,thumb_middle_x,thumb_middle_y,thumb_tip_x,thumb_tip_y,...,ring_tip_y,pinky_base_x,pinky_base_y,pinky_low_x,pinky_low_y,pinky_mid_x,pinky_mid_y,pinky_tip_x,pinky_tip_y,labels
1746,0.613851,0.895410,0.680995,0.785470,0.732318,0.671934,0.774935,0.589188,0.817468,0.548507,...,0.214373,0.534951,0.621965,0.499834,0.480175,0.483154,0.404711,0.475546,0.337150,2
1747,0.613851,0.895410,0.680995,0.785470,0.732318,0.671934,0.774935,0.589188,0.817468,0.548507,...,0.214373,0.534951,0.621965,0.499834,0.480175,0.483154,0.404711,0.475546,0.337150,2
1748,0.609669,0.898326,0.676857,0.780899,0.728712,0.671116,0.769519,0.582414,0.810667,0.531051,...,0.205530,0.530176,0.617942,0.498775,0.471503,0.485508,0.390716,0.481019,0.322753,2
1749,0.609669,0.898326,0.676857,0.780899,0.728712,0.671116,0.769519,0.582414,0.810667,0.531051,...,0.205530,0.530176,0.617942,0.498775,0.471503,0.485508,0.390716,0.481019,0.322753,2
1750,0.611743,0.913301,0.678656,0.796548,0.725939,0.669819,0.763438,0.574065,0.803300,0.522222,...,0.209425,0.528533,0.634477,0.493236,0.492139,0.477927,0.409855,0.472337,0.333683,2


In [8]:
# 21 points - looks normalized to me

from sklearn.externals.six import StringIO  
import pydot 


X_train, y_train, X_test, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

clf = DecisionTreeClassifier(random_state=0)
clf.fit(X, y)
print(np.mean(cross_val_score(clf, X, y, cv=10)))


0.8451928747270653


In [9]:
dot_data = StringIO()
sklearn.tree.export_graphviz(clf, out_file=dot_data)
graph = pydot.graph_from_dot_data(dot_data.getvalue()) 
graph[0].write_pdf("gesture.pdf")

In [10]:
clf2 = RandomForestClassifier(random_state=0)
clf2.fit(X, y)
print(np.mean(cross_val_score(clf2, X, y.values.ravel(), cv=10)))


/Users/Me/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


0.8778563699110163


In [ ]:
## Normalized Data

In [18]:
# training_df_norm = pd.read_csv('/Users/Me/Desktop/assistive_mouse/data.txt', sep=",")
# training_df_norm[["42"]] = training_df_norm[["42"]].astype("int")
# X_norm = training_df_norm.drop(columns =["42"])

# y_norm = training_df_norm[["42"]].astype(int)

X_norm, y_norm = generateTrain('/Users/Me/Desktop/assistive_mouse/data.txt')
# training_df_norm.columns = ['wrist_x', 'wrist_y', 
#                        "side_x", "side_y",
                       
#                        "thumb_base_x", "thumb_base_y", 
#                        "thumb_middle_x", "thumb_middle_y",
#                        "thumb_tip_x", "thumb_tip_y",
                       
#                        "index_base_x", "index_base_y",
#                        "index_low_x", "index_low_y",
#                        "index_mid_x", "index_mid_y",
#                        "index_tip_x", "index_tip_y",
                       
#                        "middle_base_x", "middle_base_y", 
#                        "middle_low_x", "middle_low_y",
#                        "middle_mid_x", "middle_mid_y",
#                        "middle_tip_x", "middle_tip_y",
                      
#                        "ring_base_x", "ring_base_y",
#                        "ring_low_x", "ring_low_y",
#                        "ring_mid_x", "ring_mid_y",
#                        "ring_tip_x", "ring_tip_y",
                      
#                        "pinky_base_x", "pinky_base_y",
#                        "pinky_low_x", "pinky_low_y",
#                        "pinky_mid_x", "pinky_mid_y",
#                        "pinky_tip_x", "pinky_tip_y", 
                       
#                        "labels"]
display(training_df_norm.tail(5))

,wrist_x,wrist_y,side_x,side_y,thumb_base_x,thumb_base_y,thumb_middle_x,thumb_middle_y,thumb_tip_x,thumb_tip_y,...,ring_tip_y,pinky_base_x,pinky_base_y,pinky_low_x,pinky_low_y,pinky_mid_x,pinky_mid_y,pinky_tip_x,pinky_tip_y,labels
1746,0.613851,0.895410,0.680995,0.785470,0.732318,0.671934,0.774935,0.589188,0.817468,0.548507,...,0.214373,0.534951,0.621965,0.499834,0.480175,0.483154,0.404711,0.475546,0.337150,2
1747,0.613851,0.895410,0.680995,0.785470,0.732318,0.671934,0.774935,0.589188,0.817468,0.548507,...,0.214373,0.534951,0.621965,0.499834,0.480175,0.483154,0.404711,0.475546,0.337150,2
1748,0.609669,0.898326,0.676857,0.780899,0.728712,0.671116,0.769519,0.582414,0.810667,0.531051,...,0.205530,0.530176,0.617942,0.498775,0.471503,0.485508,0.390716,0.481019,0.322753,2
1749,0.609669,0.898326,0.676857,0.780899,0.728712,0.671116,0.769519,0.582414,0.810667,0.531051,...,0.205530,0.530176,0.617942,0.498775,0.471503,0.485508,0.390716,0.481019,0.322753,2
1750,0.611743,0.913301,0.678656,0.796548,0.725939,0.669819,0.763438,0.574065,0.803300,0.522222,...,0.209425,0.528533,0.634477,0.493236,0.492139,0.477927,0.409855,0.472337,0.333683,2


In [19]:
X_train_norm, y_train_norm, X_test_norm, y_test_norm = train_test_split(X_norm, y_norm, test_size=0.3, random_state=42)

clf_norm = DecisionTreeClassifier(random_state=0)
clf_norm.fit(X_norm, y_norm)
print(np.mean(cross_val_score(clf_norm, X_norm, y_norm, cv=10)))


0.8095240891000681


In [20]:
dot_data = StringIO()
sklearn.tree.export_graphviz(clf_norm, out_file=dot_data)
graph = pydot.graph_from_dot_data(dot_data.getvalue()) 
graph[0].write_pdf("gesture_normalized.pdf")

In [22]:
clf2_norm = RandomForestClassifier(random_state=0)
clf2_norm.fit(X_norm, y_norm)
print(np.mean(cross_val_score(clf2_norm, X_norm, y_norm.ravel(), cv=10)))

0.8677788298784792
